In [1]:
import torch 
import torch.nn as nn
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel,AutoTokenizer, AutoModel
from tqdm import tqdm
from flair.embeddings import WordEmbeddings
from sentence_transformers import SentenceTransformer
import tensorflow as tf 
import os
import json
import pyarrow
import pandas as pd

c:\Users\satya\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650 Ti'

In [3]:
import pandas as pd

data = pd.read_table('fra.txt', header= None)
data.rename(columns= {0: 'English', 1: 'French', 2: 'Citation'}, inplace= True)
data = data[:100000]
data

,English,French,Citation
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
99995,He's good at telling jokes.,Il est bon pour raconter des blagues.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
99996,He's handsome and charming.,Il est beau et a du charme.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
99997,He's much younger than Tom.,Il est beaucoup plus jeune que Tom.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
99998,He's my old drinking buddy.,C'est mon vieux compagnon de beuverie.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [4]:
pattern = r"[!'#$%&()*+,-./:;<=>?@[\]^`{|}~“”‘’«»‹›„‚–—…·•¡¿’\"\']"

eng_sent, french_sent = [], []

for e in range(len(data['English'])):
    eng_sent.append(re.sub(pattern, "", data['English'][e]))
    french_sent.append(re.sub(pattern, "", data['French'][e]))
#eng_sent[229801]
eng_sent

['Go',
 'Go',
 'Go',
 'Go',
 'Hi',
 'Hi',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Run',
 'Who',
 'Wow',
 'Wow',
 'Wow',
 'Duck',
 'Duck',
 'Duck',
 'Fire',
 'Help',
 'Hide',
 'Hide',
 'Jump',
 'Jump',
 'Stop',
 'Stop',
 'Stop',
 'Wait',
 'Wait',
 'Wait',
 'Wait',
 'Wait',
 'Wait',
 'Wait',
 'Begin',
 'Begin',
 'Go on',
 'Go on',
 'Go on',
 'Hello',
 'Hello',
 'Hello',
 'Hello',
 'Hello',
 'Hello',
 'I see',
 'I see',
 'I try',
 'I won',
 'I won',
 'I won',
 'Oh no',
 'Relax',
 'Relax',
 'Relax',
 'Relax',
 'Relax',
 'Relax',
 'Relax',
 'Relax',
 'Relax',
 'Relax',
 'Relax',
 'Relax',
 'Smile',
 'Smile',
 'Smile',
 'Sorry',
 'Attack',
 'Attack',
 'Attack',
 'Attack',
 'Buy it',
 'Buy it',
 'Buy it',
 'Buy it',
 'Cheers',
 'Cheers',
 'Cheers',
 'Cheers',
 'Eat it',
 'Eat it',
 'Exhale',
 'Get up',
 'Get up',
 'Get up',
 'Go now',
 'Go now',
 'Go now',
 'Got it',
 'Got it',
 'Got it',
 'Got it',
 'Got it',
 

In [5]:
print(len(eng_sent))
print(len(french_sent))

100000
100000


In [6]:
print(100 - (len(set(eng_sent))/len(eng_sent))*100)
print(100 - (len(set(french_sent))/len(french_sent))*100)

34.038
13.204000000000008


In [7]:
eng_sent_unique = list(set(eng_sent))
french_sent_unique = list(set(french_sent))

In [8]:
sent_model = SentenceTransformer('all-MiniLM-L6-v2')
bert_small_tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-small-uncased")
bert_small_model = AutoModel.from_pretrained("nlpaueb/legal-bert-small-uncased")
bert_tiny_tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
Bert_tiny_model = AutoModel.from_pretrained("prajjwal1/bert-tiny")

In [9]:
Bert_tiny_model.device

device(type='cpu')

In [10]:
def text_embedding2(batch_tokens, max_len, model, tokenizer):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #model = model.to(device)

    # Reduce the batch size if the input is too large for GPU memory
    batch_size = len(batch_tokens)
    max_batch_size = 32  # You can adjust this value based on your GPU memory capacity

    while batch_size > max_batch_size:
        batch_tokens = batch_tokens[:batch_size // 2]  # Halve the batch size
        batch_size = len(batch_tokens)

    batch_padded_tokens = [tokens + [tokenizer.pad_token_id] * (max_len - len(tokens))
                           for tokens in batch_tokens]

    tokens_tensor = torch.tensor(batch_padded_tokens)
    with torch.no_grad():
        output = model(tokens_tensor)
        embeddings = output.last_hidden_state
        torch.cuda.empty_cache()

    return embeddings

In [14]:

def get_embeddings(max_length,batch_size,tokens,model,tokenizer):

    embedding_trans = []
    for i in tqdm(range(0, len(tokens), batch_size), "Embedding", colour= "green"):
        batch_token = tokens[i : i+batch_size]
        embedding_trans.extend(text_embedding2(batch_token,max_length,model,tokenizer))

    return embedding_trans

In [12]:
torch.cuda.empty_cache()
english_tokens = [bert_tiny_tokenizer.encode(text,add_special_tokens = True,padding='max_length',max_length=104) for text in eng_sent]
french_token = [bert_tiny_tokenizer.encode(text,add_special_tokens = True,padding='max_length',max_length=104) for text in french_sent]
English_embeddings = get_embeddings(max_length=104,batch_size=32,tokens=english_tokens,model=Bert_tiny_model,tokenizer=bert_tiny_tokenizer)
torch.cuda.empty_cache()
print('----------------------English embededding done -------------------')
French_embeddings = get_embeddings(max_length=104,batch_size=32,tokens=french_token,model=Bert_tiny_model,tokenizer=bert_tiny_tokenizer)
print('----------------------French embededding done -------------------')
torch.cuda.empty_cache()

Embedding: 100%|██████████| 3125/3125 [01:33<00:00, 33.39it/s]


----------------------English embededding done -------------------


Embedding: 100%|██████████| 3125/3125 [01:34<00:00, 32.93it/s]

----------------------French embededding done -------------------


In [13]:
len(English_embeddings)

100000

In [103]:
batch_size = 100
#english_tensor_stacked = torch.stack(English_embeddings,dim=0)
#french_tensor_stacked = torch.stack(French_embeddings)
num_batches = len(English_embeddings) // batch_size
for i in range(num_batches):
    batched_tensor = torch.stack([torch.stack(English_embeddings[i*batch_size:(i+1)*batch_size])]).cuda()
    torch.cuda.empty_cache()
print(batched_tensor.shape)

torch.Size([1, 100, 104, 128])


In [20]:

def get_batched_embeddings(batch_size,embeddings,steps_per_epoch):


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    num_batches = len(embeddings) // batch_size
    remainder = len(embeddings) % batch_size

    # Define a function to process a batch
    def process_batch(batch):
        return torch.stack(batch)
    
    max_batch_size = 100  # You can adjust this value based on your GPU memory capacity

    # Create a list to store individual batches
    batch_list = []

    for i in range(num_batches):
        if i%steps_per_epoch == 0:
            print(f'Computing {i}th batch')
        batch_start = i * batch_size
        batch_end = (i + 1) * batch_size
        with torch.no_grad():
            batch = embeddings[batch_start:batch_end]
            while batch_size > max_batch_size:
                batch = batch[:batch_size // 2]  # Halve the batch size
                batch_size = len(batch)
            processed_batch = process_batch(batch)
            batch_list.append(processed_batch)
            torch.cuda.empty_cache()

    # Handle the remainder
    if remainder > 0:
        last_batch = embeddings[-remainder:]
        processed_last_batch = process_batch(last_batch)
        batch_list.append(processed_last_batch)
        torch.cuda.empty_cache()

    # Stack the batches along a new dimension
    batched_tensor = torch.stack(batch_list, dim=0)

    return batched_tensor
English_batched_embeddings = get_batched_embeddings(100,English_embeddings,100)
English_batched_embeddings.size()

Computing 0th batch
Computing 100th batch
Computing 200th batch
Computing 300th batch
Computing 400th batch
Computing 500th batch
Computing 600th batch
Computing 700th batch
Computing 800th batch
Computing 900th batch


RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 5324800000 bytes.

In [21]:
def get_batched_embeddings(batch_size, embeddings, steps_per_epoch):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    num_batches = len(embeddings) // batch_size
    remainder = len(embeddings) % batch_size

    # Define a function to process a batch
    def process_batch(batch):
        return torch.stack(batch)

    max_batch_size = 100  # You can adjust this value based on your GPU memory capacity

    # Create a list to store individual batches
    batch_list = []

    for i in range(num_batches):
        if i % steps_per_epoch == 0:
            print(f'Computing {i}th batch')

        batch_start = i * batch_size
        batch_end = (i + 1) * batch_size
        with torch.no_grad():
            batch = embeddings[batch_start:batch_end]
            batch_size = len(batch)  # Reset batch_size for the inner loop

            while batch_size > max_batch_size:
                sub_batch = batch[:max_batch_size]
                processed_sub_batch = process_batch(sub_batch)
                batch_list.append(processed_sub_batch)
                torch.cuda.empty_cache()

                batch = batch[max_batch_size:]
                batch_size = len(batch)

            processed_batch = process_batch(batch)
            batch_list.append(processed_batch)
            torch.cuda.empty_cache()

    # Handle the remainder
    if remainder > 0:
        last_batch = embeddings[-remainder:]
        processed_last_batch = process_batch(last_batch)
        batch_list.append(processed_last_batch)
        torch.cuda.empty_cache()

    # Stack the batches along a new dimension
    batched_tensor = torch.stack(batch_list, dim=0).to(device)

    return batched_tensor
English_batched_embeddings = get_batched_embeddings(100,English_embeddings,100)
English_batched_embeddings.size()

Computing 0th batch
Computing 100th batch
Computing 200th batch


: 

In [101]:
x = torch.randn(16,4)
x = torch.tile(x,(32,1,1))
x.size()

torch.Size([32, 16, 4])

In [102]:
import numpy as np

def positional_encoding(embedding_vectors):

    batches,batch_size,max_length, d_model = embedding_vectors.size()
    
    even_i = torch.arange(0 , d_model , 2).float()
    even_denominator = torch.pow(10000, even_i/d_model)
    odd_i = torch.arange(1 , d_model , 2).float()
    odd_denominator = torch.pow(10000, (odd_i -1)/d_model)

    positions = torch.arange(max_length,dtype=torch.float).reshape(max_length,1)

    even_pe = torch.sin(positions/even_denominator)
    odd_pe = torch.sin(positions/even_denominator)
    stacked = torch.stack([even_pe , odd_pe] , dim  = 2)
    PE = torch.flatten(stacked,start_dim=1,end_dim=2)
    PE = torch.tile(PE,(batch_size,1,1))
    test_list=[]

    for i in range(batches):
        test_list.append(embedding_vectors[i] + PE)
    test_list = torch.stack(test_list)
    return test_list

# Example usage:
l = positional_encoding(batched_tensor)


In [103]:
l.size()

torch.Size([100, 1000, 104, 128])

In [31]:
sample = english_tokens[0]
len(sample)

104

In [34]:
batch_padded_tokens = [sample + [bert_tiny_tokenizer.pad_token_id 
                                 for i in range(104 - len(sample))]
                      ]
batch_padded_tokens
tokens_tensor = torch.tensor(batch_padded_tokens)
with torch.no_grad():
    output = Bert_tiny_model(tokens_tensor)
    embeddings = output.last_hidden_state
embeddings.size()

torch.Size([1, 104, 128])

In [13]:
path = "embedding_files"
if not os.path.exists(path):
   os.makedirs(path)
   print("The new directory is created!")
json_path = os.path.join(os.getcwd(),path)
json_path

'd:\\projects\\Machine-Translation\\embedding_files'

# writing the english embeddings in a json file and uploading it for time saving

In [17]:
eng_embedding_dict = {}
eng_embeddings_parquet_path = 'eng_embeds.parquet'
for i in range(len(eng_embeddings)):
    eng_embedding_dict[i] = eng_embeddings[i].tolist()
#eng_embedding_dict[1]
#eng_embedding_dict_json_object = json.dumps(eng_embedding_dict, indent = 4)
eng_embedding_df = pd.DataFrame(eng_embedding_dict)
eng_embedding_df.to_parquet(os.path.join(json_path,eng_embeddings_parquet_path))
#with open(os.path.join(json_path,eng_embeddings_json_path), "w") as outfile:
    #outfile.write(eng_embedding_dict_json_object)

# writing the french embeddings for time saving

In [18]:
french_embedding_dict = {}
french_embeddings_parquet_path = 'french_embeds.parquet'
for i in range(len(eng_embeddings)):
    french_embedding_dict[i] = fr_embeddings[i].tolist()
#eng_embedding_dict[1]
#french_embedding_dict_json_object = json.dumps(french_embedding_dict, indent = 4) 
fr_embedding_df = pd.DataFrame(french_embedding_dict)
fr_embedding_df.to_parquet(os.path.join(json_path,french_embeddings_parquet_path))
#with open(os.path.join(json_path,french_embeddings_json_path), "w") as outfile:
    #outfile.write(french_embedding_dict_json_object)